# 活用まとめ
1. 引数チェック
2. キャッシュ
3. プロキシ
4. コンテキストプロバイダ

In [1]:
rpc_info= {}
def xmlrpc(in_ = (),out = (type(None),)):
    def _xmlrpc(function):
        # 引数情報の登録
        func_name = function.__name__
        rpc_info[func_name]=(in_,out)
        
        def _check_types(elements,types):
            """型をチェックするサブ関数"""
            if len(elements)!= len(types):
                raise TypeError("引数の個数を間違えています")
            typed =enumerate(zip(elements,types))
            for index,couple in typed:
                arg,of_the_right_type=couple
                if isinstance(arg,of_the_right_type):

                    continue
                raise TypeError("引数#%dは%s型である必要があります"%(index,of_the_right_type))
                
        def __xmlrpc(*args):
            # 入力チェック
            checkable_args = args[1:]
            _check_types(checkable_args,in_)
            # 関数の実行
            res = function (*args)
            # 出力値のチェック
            if not type(res)in (tuple,list):
                checkable_res = (res,)
            else:
                checkable_res = res
            _check_types(checkable_res,out)
            # 関数と型のチェックが成功
            return res
        return __xmlrpc
    return _xmlrpc

In [2]:
class RPCView:
    @xmlrpc((int,int))
    def meth1(self,int1,int2):
        print("%dと%dを受け取りました"%(int1,int2))
    @xmlrpc((str,),(int,))
    def meth2(self,phrase):
        print("%sを受け取りました"%phrase)
        return 12
    

In [3]:
rpc_info

{'meth1': ((int, int), (NoneType,)), 'meth2': ((str,), (int,))}

In [4]:
my = RPCView()

In [5]:
my.meth1(1,2)

1と2を受け取りました


In [6]:
my.meth2(2)

TypeError: 引数#0は<class 'str'>型である必要があります

In [7]:
# キャッシュ（メモ化 MEMONIZE）
# 出力が内部状態の影響を受けないときに有効．，計算が高付加な関数など
import time
import hashlib
import pickle

cash = {}

def is_obsolete(entry,duration):
    return time.time() -entry["time"]>duration

def compute_key(function,args,kw):
    key= pickle.dumps((function.__name__,args,kw))
    return hashlib.sha1(key).hexdigest()

def memoize(duration = 10):
    def _memoize(function):
        def __memoize(*args,**kw):
            key = compute_key(function,args,kw)
            #計算済みか否か
            if (key in cash and not is_obsolete(cash[key],duration)):
                print("キャッシュ済みの値を取得")
                return cash[key]["value"]
            # 計算
            result = function(*args,**kw)
            # 結果の保存
            cash[key]={"value" :result,"time":time.time()}
            return result
        return __memoize
    return _memoize

In [8]:
@memoize()
def very_vomplex_complex_stuff(a,b):
    return a + b

In [27]:
very_vomplex_complex_stuff(2,4)

6

In [28]:
very_vomplex_complex_stuff(2,4)

キャッシュ済みの値を取得


6

In [29]:
# プロキシ
# 関数のタグ付けやグローバルな仕組みへの登録
class User(object):
    def __init__(self, roles):
        self.roles = roles

class Unauthorized(Exception):
    pass

def protect (role):
    def _protect(function):
        def __protect(*arg,**kw):
            user= globals().get("user")
            # globals() グローバル変数を辞書形式で取得
            if user is None or role not in user.roles:
                raise Unauthorized("ひ み つ")
            return function(*arg,**kw)
        return __protect
    return _protect
## Django には関数アクセスを安全にすデコレータがある

In [35]:
tarek = User(("admin","user"))
bill =User(("user",))
class MySecrets(object):
    @protect("admin")
    def waffle_recipe(self):
        print("バター：１トン")

In [38]:
these_are = MySecrets()
user = tarek
these_are.waffle_recipe()

バター：１トン


In [39]:
user = bill
these_are.waffle_recipe()

Unauthorized: ひ み つ

In [41]:
# コンテキストプロバイダ
# 関数が正しい実行コンテキスト内で実行されることを保証したり，関数前後の処理を行ったりする
# スレッド間でのデータ共有のため，複数のスレッドがらアクセスされてもデータが保護されていることを保証するためにロックを使用する
# With 文の方が一般的か
from threading import RLock
lock=RLock()

def scyncronized(function):
    def _scyncronized(*args,**kw):
        lock.acquire()
        try:
            return function(*args,**kw)
        finally:
            lock.release()
    return _scyncronized

@scyncronized
def thread_safe():
    pass
        

In [47]:
class MyClass:
    """My Doc Strings"""
    pass

my = MyClass()
def addto(instance):
    def _addto(f):
        import types
        f =types.MethodType(f,instance)
        setattr(instance,f.__name__,f)
        # インスタンスへ関数fの名前とそのメソッドを追加する
        return f
    return _addto

@addto(my)
def print_decstrings(self):
    print(self.__doc__)
    


In [48]:
my.print_decstrings()

My Doc Strings


In [55]:
def print_args(function):
    def _print_args(*args,**kw):
        print(function.__name__,args,kw)
        return function(*args,**kw)
    return _print_args
 
@print_args
def my_function(a,b,c):
    print (a+b,c*2)
my_function(5,4,c="key")

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(a+b,c*2)? (<ipython-input-55-410658abc840>, line 9)

In [56]:
def onexit(function):
    import atexit
    atexit.register(function)
    return function
@onexit
def post_function():
    print("プロセスが終了します")

In [57]:
post_function()

プロセスが終了します
